# Track length per contract region (and year)

## Read input data

In [1]:
import pandas as pd # type: ignore

# Step 1: Load the Excel file 
excel_file_path = "../Python matching/raw_data/Kontraktsområde, kontrakt, bandel.xlsx"
contract_df = pd.read_excel(excel_file_path, sheet_name="Kontraktsområden 2011-2023")

In [2]:
# rename column Kontraktsområdesnamn [kan behöva justeras] to Kontraktsområdesnamn
contract_df.rename(columns={'Kontraktsområdesnamn [kan behöva justeras]': "Kontraktsområdesnamn"}, inplace=True)
# rename column Kontraktsnummer \n(Diarienummer) to Kontraktsnummer
contract_df.rename(columns={"Kontraktsnummer \n(Diarienummer)": "Kontraktsnummer"}, inplace=True)

In [3]:
print(contract_df.columns)

Index(['År', 'Kontraktsområdesnamn', 'Fiktivt områdesnummer',
       'Kontraktsnummer', 'Starttid', 'Sluttid', 'Bandel', 'Del av bandel',
       'Övergång till nytt område', 'Övergång från gammalt område',
       'Kommentar', 'Antagande'],
      dtype='object')


In [4]:
excel_file_path = "../Python matching/raw_data/BIS-data 2024-01-09 - Bandel, plats och förbindelselinje, alla spår.xlsx"
bis_df = pd.read_excel(excel_file_path)
bis_df.columns

Index(['Banlangd', 'BdlNr', 'Bandel', 'Plats_sign', 'Plats', 'Forbind',
       'Spår_huvud_sido', 'UNE', 'Spårnummer'],
      dtype='object')

## Processing

In [5]:
# # remove rows from contract_df where Övergång till nytt område is 1
# print(len(contract_df))
# contract_df = contract_df[contract_df["Övergång till nytt område"] != 1]
# print(len(contract_df))

In [6]:
# if duplicates (same 'År', 'Bandel') are found in contract_df, keep the one with Övergång från gammalt område = 1
# sort by 'Övergång från gammalt område' and drop duplicates
print(len(contract_df))
contract_df = contract_df.sort_values('Övergång från gammalt område').drop_duplicates(subset=['År', 'Bandel'], keep='first')
# reset the order of the rows
contract_df = contract_df.sort_index()
print(len(contract_df))

3360
3116


## Track length (nhsp)

In [7]:
print(bis_df.columns)

Index(['Banlangd', 'BdlNr', 'Bandel', 'Plats_sign', 'Plats', 'Forbind',
       'Spår_huvud_sido', 'UNE', 'Spårnummer'],
      dtype='object')


```markdown
To add the total track length on main tracks (normalhuvudspår, nhsp), we will use the columns `BdlNr` and `Spår_huvud_sido` (equal to 'nhsp') and `Banlangd` from `bis_df`. 

For each year, we will first add the column `Kontraktsområdesnamn` to `bis_df_processed` based on `BdlNr` using `contract_df`. This will give us the matching between `Kontraktsområdesnamn` and `Bandel` for a given year.
```

In [8]:
# first step is to filter bis_df to include only rows where Spår_huvud_sido is 'nhsp'
bis_df_nhsp = bis_df[bis_df['Spår_huvud_sido'] == 'nhsp']

In [9]:
# second, group by BdlNr and sum Banlangd
bis_df_nhsp_grouped = bis_df_nhsp.groupby('BdlNr').agg({'Banlangd': 'sum'}).reset_index()
# convert to km
bis_df_nhsp_grouped['Banlangd'] = bis_df_nhsp_grouped['Banlangd'] / 1000 # convert to km

In [10]:
# Merge bis_df_nhsp with contract_df to get Kontraktsområdesnamn
bis_df_processed = bis_df_nhsp_grouped.merge(contract_df[['År', 'Bandel', 'Kontraktsområdesnamn']], 
                                     left_on=['BdlNr'], 
                                     right_on=['Bandel'], 
                                     how='right')

In [11]:
# Remove rows where Kontraktsområdesnamn is NaN
#bis_df_processed = bis_df_processed[bis_df_processed['Banlangd'].notna()]

# Calculate the total track length for each year and Kontraktsområdesnamn
banlangd_kontrakt = bis_df_processed.groupby(['År', 'Kontraktsområdesnamn', 'Bandel'])['Banlangd'].sum().reset_index()
# Rename columns for clarity
banlangd_kontrakt.rename(columns={'Banlangd': 'Banlangd_nhsp'}, inplace=True)

## Track length (all spår)

We do the same as for nhsp to calculate the length to all tracks.

In [12]:
# second, group by BdlNr and sum Banlangd
bis_df_grouped = bis_df.groupby('BdlNr').agg({'Banlangd': 'sum'}).reset_index()
# convert to km
bis_df_grouped['Banlangd'] = bis_df_grouped['Banlangd'] / 1000 # convert to km

In [13]:
# Merge bis_df_nhsp with contract_df to get Kontraktsområdesnamn
bis_df_alla_processed = bis_df_grouped.merge(contract_df[['År', 'Bandel', 'Kontraktsområdesnamn']], 
                                     left_on=['BdlNr'], 
                                     right_on=['Bandel'], 
                                     how='right')

In [14]:
# Remove rows where Kontraktsområdesnamn is NaN
#bis_df_alla_processed = bis_df_alla_processed[bis_df_alla_processed['Banlangd'].notna()]

# Calculate the total track length for each year and Kontraktsområdesnamn
banlangd_kontrakt_alla = bis_df_alla_processed.groupby(['År', 'Kontraktsområdesnamn', 'Bandel'])['Banlangd'].sum().reset_index()
# Rename columns for clarity
banlangd_kontrakt_alla.rename(columns={'Banlangd': 'Banlangd_alla'}, inplace=True)

## Export all the data

In [ ]:
# merge banlangd_kontrakt and banlangd_kontrakt_alla based on year and Kontraktsområdesnamn
banlangd_kontrakt_merged = banlangd_kontrakt.merge(banlangd_kontrakt_alla,
                                                   on=['År', 'Kontraktsområdesnamn','Bandel'],
                                                    how='inner')

# export to excel and csv
banlangd_kontrakt_merged.to_excel('./exported_data_regression/banlangd_kontrakt_merged_bandelnr.xlsx', index=False)
banlangd_kontrakt_merged.to_csv('./exported_data_regression/banlangd_kontrakt_merged_bandelnr.csv', index=False)